In [1]:
pip install plyer requests schedule

In [1]:
pip install PyPDF2 plyer

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install python-docx


  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1


In [ ]:
import docx
import random
import time
import schedule
import tkinter as tk
from tkinter import messagebox

# Global variable to store questions
questions = []

def extract_questions_from_docx(file_path):
    global questions
    doc = docx.Document(file_path)
    
    question_data = {}
    for para in doc.paragraphs:
        line = para.text.strip()
        if not line:  # Skip empty lines
            continue
        
        if line.startswith("Answer:"):
            # Extract correct answer
            correct_answer = line.split(': ')[1].strip().lower()  # Get answer option (e.g., 'a')
            question_data['correct_answer'] = correct_answer
            
            # Ensure all keys are present before appending
            if 'question' in question_data and 'options' in question_data:
                questions.append(question_data)  # Add question data to list
            
            question_data = {}  # Reset for next question
        
        # Check for options like 'a)', 'b)', etc.
        elif len(line) > 0 and (line[0].isalpha() and ')' in line):  
            if 'options' not in question_data:
                question_data['options'] = []  # Initialize options list
            
            option_letter = line[0].lower()  # Get option letter (e.g., 'a')
            option_text = line[3:]  # Get option text (after 'a) ')
            question_data['options'].append(option_text)  # Add option text to list
        
        # Check for questions starting with a number followed by a dot (e.g., "1. ")
        elif len(line) > 0 and line[0].isdigit() and '.' in line:  
            if 'question' not in question_data:
                question_data['question'] = line  # First line with number is the question

def show_notification():
    if not questions:
        print("No questions available.")
        return
    
    question_data = random.choice(questions)
    
    if 'question' not in question_data or 'options' not in question_data or 'correct_answer' not in question_data:
        return
    
    question = question_data['question']
    correct_answer_index = ord(question_data['correct_answer']) - ord('a')  # Convert letter to index (0-3)
    correct_answer_text = question_data['options'][correct_answer_index]  # Get correct answer text
    options = question_data['options']
    
    # Create the main window for the notification
    root = tk.Tk()
    root.title("Programing  Questions")
    
    # Create a label for the question
    label = tk.Label(root, text=question, padx=100, pady=100)
    label.pack()

    # Function to handle user answer selection
    def submit_answer(selected_option):
        if selected_option == correct_answer_text:  # Compare with correct answer text
            messagebox.showinfo("Result", "Correct! 🎉")
        else:
            messagebox.showinfo("Result", f"Incorrect! The correct answer is: {correct_answer_text}.")
        root.destroy()  # Close the window after answering
    
    # Shuffle options for randomness
    random.shuffle(options)

    # Create buttons for each option
    for option in options:
        option_button = tk.Button(root, text=option, command=lambda opt=option: submit_answer(opt))
        option_button.pack(pady=5)

    skip_button = tk.Button(root, text="Skip Question", command=lambda: [root.destroy()])
    skip_button.pack(pady=5)

    # Start the GUI loop
    root.mainloop()

# Function to run every 2 minutes
def job():
    if not questions:
        print("No questions loaded. Please load the questions from a Word document.")
        return
    
    show_notification()

if __name__ == "__main__":
    pdf_file_path = input("Enter the path to the Word file: ").strip()
    
    extract_questions_from_docx(pdf_file_path)
    
    # Schedule the job every 2 minutes
    schedule.every(2).minutes.do(job)

    print("Notification service started. Press Ctrl+C to exit.")

    # Keep the script running
    while True:
        schedule.run_pending()
        time.sleep(1)

Enter the path to the Word file: C:\Users\kames\Downloads\Question.docx
Notification service started. Press Ctrl+C to exit.
